In [77]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [78]:
RANDOM_STATE = 42

In [79]:
df = pd.read_csv('./lenta_df.csv')

In [80]:
nltk.download('stopwords')
nltk.download('punkt_tab')
russian_stopwords = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gulfik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Gulfik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [81]:
def preprocess_text(text):
    """
    Делаем токенизацию и удаляем стоп-слова с помощью библиотеки nltk
    :param text:
    :return токены:
    """
    tokens = word_tokenize(text, language='russian')
    filtered_tokens = [word for word in tokens if word.isalnum() and word.lower() not in russian_stopwords]
    return ' '.join(filtered_tokens)


df['text'] = df['text'].dropna().apply(preprocess_text)
cleaned_df = df.copy()
cleaned_df.shape

(22500, 3)

In [82]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(cleaned_df['text'], cleaned_df['bloc'], test_size=0.2,
                                                    random_state=RANDOM_STATE)

Используем Tf-idf со случайными параметрами. Выбрал его вместо CountVectorizer, потому что посчитал, что tfidf учитывает редкость слов в корпусе документов.

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

Обучем несколько моделей со случайными параметрами

In [84]:
from sklearn.linear_model import LogisticRegression

model_lg = LogisticRegression(max_iter=1000, solver='lbfgs', random_state=RANDOM_STATE)
model_lg.fit(X_train_tfidf, y_train)
model_lg_pred = model_lg.predict(X_test_tfidf)

In [85]:
from sklearn.naive_bayes import MultinomialNB

model_nb = MultinomialNB()
model_nb.fit(X_train_tfidf, y_train)
model_nb_pred = model_nb.predict(X_test_tfidf)

In [98]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=200, max_depth=30)
model_rf.fit(X_train_tfidf, y_train)
model_rf_pred = model_rf.predict(X_test_tfidf)

In [87]:
from sklearn.svm import LinearSVC

model_svm = LinearSVC(random_state=RANDOM_STATE, C=1)
model_svm.fit(X_train_tfidf, y_train)
model_svm_pred = model_svm.predict(X_test_tfidf)

In [94]:
from sklearn.tree import DecisionTreeClassifier

model_tree = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=30)
model_tree.fit(X_train_tfidf, y_train)
model_tree_pred = model_tree.predict(X_test_tfidf)

In [99]:
from sklearn.metrics import classification_report


print(f'Logistic regression report: {classification_report(y_test, model_lg_pred)}')
print(f'NB report: {classification_report(y_test, model_nb_pred)}')
print(f'Random forest report: {classification_report(y_test, model_rf_pred)}')
print(f'SVM report: {classification_report(y_test, model_svm_pred)}')
print(f'Decision tree report: {classification_report(y_test, model_tree_pred)}')
# print(f'XGB classifier report: {classification_report(y_test, model_xgb_pred)}')

Logistic regression report:               precision    recall  f1-score   support

           0       0.80      0.86      0.83       469
           1       0.88      0.87      0.88       498
           2       0.94      0.96      0.95       472
           3       0.91      0.93      0.92       543
           4       0.99      1.00      1.00       474
           5       0.99      0.99      0.99       561
           6       0.84      0.77      0.81       489
           7       0.99      0.97      0.98       477
           8       0.95      0.94      0.94       517

    accuracy                           0.92      4500
   macro avg       0.92      0.92      0.92      4500
weighted avg       0.92      0.92      0.92      4500

NB report:               precision    recall  f1-score   support

           0       0.78      0.69      0.73       469
           1       0.78      0.82      0.80       498
           2       0.83      0.96      0.89       472
           3       0.81      0.93      

Посмотрим на roc_auc Logistic Regression

In [109]:
from sklearn.metrics import roc_auc_score

model_lg_proba = model_lg.predict_proba(X_test_tfidf)
roc_auc = roc_auc_score(y_test, model_lg_proba, multi_class='ovr')

print('ROC AUC для log reg:', roc_auc)

ROC AUC для log reg: 0.9939003199774606


Из репортов видно, что лучше всего с задачей справились линейная SVM и Logistic Regression. Скорее всего они справились лучше всего, так как я использовал tf-idf, и после этой обработки признаков, данные стали линейно разделимы.

Попрубем подобрать параметры для Logistic Regression, и, например, для Random Forest:

In [113]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('logreg', LogisticRegression(random_state=RANDOM_STATE))
])

param_grid = {
    # Параметры TF-IDF
    'tfidf__max_features': [5000, 10000, 20000],     # Максимальное количество признаков (токенов)
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],  # Диапазон n-грамм (униграммы, биграммы, триграммы)
    'tfidf__min_df': [1, 2, 5],                      # Минимальная частота документа для включения токена
    'tfidf__max_df': [0.8, 0.9, 1.0],                # Максимальная доля документов, содержащих токен

    # Параметры Logistic Regression
    'logreg__C': [0.01, 0.1, 1, 10, 100],
    'logreg__solver': ['lbfgs', 'liblinear'],
    'logreg__penalty': ['l2'],
    'logreg__max_iter': [100, 500, 1000]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring='accuracy',
    verbose=2,
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

print('Лучшие параметры Logistic Regression:', grid_search.best_params_)
print('Лучшая точность на валидации:', grid_search.best_score_)

best_lg_pipeline = grid_search.best_estimator_
lg_pred = best_lg_pipeline.predict(X_test)

print(classification_report(y_test, lg_pred))

Fitting 3 folds for each of 2430 candidates, totalling 7290 fits
Лучшие параметры Logistic Regression: {'logreg__C': 100, 'logreg__max_iter': 100, 'logreg__penalty': 'l2', 'logreg__solver': 'liblinear', 'tfidf__max_df': 1.0, 'tfidf__max_features': 20000, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2)}
Лучшая точность на валидации: 0.9213333333333334
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       469
           1       0.86      0.88      0.87       498
           2       0.94      0.95      0.95       472
           3       0.92      0.93      0.93       543
           4       0.99      1.00      0.99       474
           5       0.99      0.99      0.99       561
           6       0.85      0.77      0.81       489
           7       0.98      0.98      0.98       477
           8       0.95      0.95      0.95       517

    accuracy                           0.92      4500
   macro avg       0.92      0.92      0.92      4

In [115]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('rf', RandomForestClassifier(random_state=RANDOM_STATE))
])

# Параметры
param_grid = {
    'tfidf__max_features': [20000],
    'tfidf__ngram_range': [(1, 2)],
    'tfidf__min_df': [5],

    'rf__n_estimators': [100, 200, 500],
    'rf__max_depth': [10, 20, 30],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring='accuracy',
    verbose=2,
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

print('Лучшие параметры RandomForest:', grid_search.best_params_)
print('Лучшая точность на валидации:', grid_search.best_score_)

best_rf_pipeline = grid_search.best_estimator_
rf_pred = best_rf_pipeline.predict(X_test)

print(classification_report(y_test, rf_pred))

Fitting 3 folds for each of 81 candidates, totalling 243 fits


C:\Users\Gulfik\Desktop\petprojects\hseContest\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Лучшие параметры RandomForest: {'rf__max_depth': 30, 'rf__min_samples_leaf': 4, 'rf__min_samples_split': 10, 'rf__n_estimators': 500, 'tfidf__max_features': 20000, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2)}
Лучшая точность на валидации: 0.8819444444444445
              precision    recall  f1-score   support

           0       0.78      0.73      0.75       469
           1       0.80      0.84      0.82       498
           2       0.86      0.94      0.90       472
           3       0.84      0.92      0.88       543
           4       0.96      0.99      0.98       474
           5       0.95      0.99      0.97       561
           6       0.90      0.73      0.81       489
           7       0.96      0.96      0.96       477
           8       0.94      0.87      0.90       517

    accuracy                           0.89      4500
   macro avg       0.89      0.89      0.89      4500
weighted avg       0.89      0.89      0.89      4500



Как видно из репорта, модели хуже всего определяют нулевой класс (тему 'Общество/Россия'). Также модели не улучшились после подбора параметров. Вероятно, это связано с тем, что данные после tf-idf уже хорошо разделимы, поэтому для улучшения можно заняться созданием новых признаков или улучшить препобработку данных.

Обучим на тестовых данных и сохраним submission:

In [118]:
test_df = pd.read_csv('./test_news.csv')
test_df['cleaned_text'] = test_df['content'].apply(preprocess_text)
y_pred_test = best_lg_pipeline.predict(test_df['cleaned_text'])
pd.DataFrame({'topic': y_pred_test}).reset_index().to_csv('submission.csv', index=False)